<a href="https://colab.research.google.com/github/alexzay07/---27-ZK.-Korenev.-Work-with-data/blob/main/%D0%A0%D0%95_27_ZK_Work_with_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#установим необходимые библиотеки
!pip install langchain
!pip install openai
!pip install chromadb
!pip install tiktoken
!pip install markdown-it-py
!pip install -qU langchain-openai
!pip install langchain-chroma

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "ключ"

In [ ]:
#разобьем текст на фрагменты
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
text = 'Благодаря появлению LLM возникла новая професссия будущего. \
Это промпт-инженер, который наилучшим образом направляет LLM на правильный путь . \
Этот специалист должен обладать творческим мышлением и аналитическими способностями, \
и желательно знанием методов машинного обучения и NLP и навыками программирования'

splitter = RecursiveCharacterTextSplitter(
    chunk_size=250,#250
    chunk_overlap=100,#100
    separators = ['. ']#
)
splited_data = splitter.split_text(text)
splited_data

['Благодаря появлению LLM возникла новая професссия будущего. Это промпт-инженер, который наилучшим образом направляет LLM на правильный путь',
 '. Это промпт-инженер, который наилучшим образом направляет LLM на правильный путь . Этот специалист должен обладать творческим мышлением и аналитическими способностями, и желательно знанием методов машинного обучения и NLP и навыками программирования']

In [ ]:
#найдем в векторной базе наиболее подходящие фрагменты
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()
from langchain_chroma import Chroma
vectordb_ct = Chroma.from_texts(splited_data, embedding=embedding)
question = "Кто такой промпт-инженер?"
docs = vectordb_ct.similarity_search(question,k=1)
docs

[Document(page_content='. Это промпт-инженер, который наилучшим образом направляет LLM на правильный путь . Этот специалист должен обладать творческим мышлением и аналитическими способностями, и желательно знанием методов машинного обучения и NLP и навыками программирования')]

In [ ]:
vectordb_ct.max_marginal_relevance_search(question,k=2, fetch_k=4)

In [ ]:
#скалярное произведение векторов. Схожесть
import numpy as np
question = 'промпт-инженер'

embedding1 = embedding.embed_query(question)
embedding2 = embedding.embed_query(docs[0].page_content)
np.dot(embedding1, embedding2)

0.8707092792942503

In [ ]:
#скалярное произведение векторов. Схожесть
import numpy as np
question = "Сколько стоят апельсины"

embedding1 = embedding.embed_query(question)
embedding2 = embedding.embed_query(docs[0].page_content)
np.dot(embedding1, embedding2)

0.7247773680430725

**Загрузка данных с файлов и сохранение в БД**

In [ ]:
!pip install langchain_community

In [ ]:
#Загрузим файлы на сервер colab
from google.colab import files
upload = files.upload()
#C:\Users\lesni\Documents\_gitprojects\llm\кейс1\main_info.md

Saving main_info.md to main_info.md


In [ ]:
#получим список загруженных файлов
!ls

main_info.md  sample_data


In [ ]:
⬇️Не используй эту команду при первой работе с кодом, так как она удалит загруженный файл⬇️

In [ ]:
!rm *.md

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import DirectoryLoader
path = ''
text_loader_kwargs={'autodetect_encoding': True}
loader = DirectoryLoader(path, glob="**/*info.md", loader_cls=TextLoader,
                         loader_kwargs=text_loader_kwargs)
docs = loader.load()
len(docs)

1

In [ ]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", "(?<=\. )"]
)
splitted_texts = splitter.split_documents(docs)
#pp(a[0].page_content)
len(splitted_texts)

12

In [ ]:
splitted_texts[1] #увеличить размер чанка

Document(page_content='## Бронирование\n\nДля бронирования дня рождения требуется предоплата 1000 рублей (переводом на карту администратора)\n\n# Услуги и локации комплекса\n\n## Игровые комнаты и локации\n\nКомплекс состоит из пяти  игровых локаций:\n- Лабиринт: Трехэтажный детский игровой лабиринт с батутами и различными препятствиями внутри.\n- Красочные сети: аттракцион состоит из множества переплетенных вручную разноцветных нейлоновых нитей, закрепленных на прочной конструкции.\n- Веревочный парк: здесь дети смогут показать свою ловкость и смелость, преодолевая различные препятствия на высоте.\n- Мультирум: игровое пространство наполнено трендовыми игрушками такие как  LEGO, Hot Wheels, Barbie, L.O.L. и многие другие. \n- Песочница: песочница с мраморным песком  для самых маленьких гостей\nПредусмотрена услуга Мультипарк, которая включает в себя следующие локации: лабиринт, красочные сети, песочница\n\n## Дополнительные услуги при групповом посещении игровых комнат на время праздн

In [ ]:
#преобразуем в вектор и сохраним БД
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
persist_directory = ''
embedding = OpenAIEmbeddings()
vectordb = Chroma.from_documents(
    documents=splitted_texts,
    embedding=embedding,
    persist_directory=persist_directory
) #создаем БД
print(vectordb._collection.count())

12


In [ ]:
vectordb.delete_collection()

In [ ]:
#загрузим бд с локалки
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)
print(vectordb._collection.count())

12


**Поиск данных в БД**

In [ ]:
def printAnswer(answers):
    for answer in answers:
        print(answer.page_content)
        print(answer.metadata)#.source
        print('-----')

question = "Где вы находитесь?"
question = "как с вами связаться?"
answers = vectordb.similarity_search(question,k=3)

printAnswer(answers)

# Общая информация о комплексе

## Контактная информация

Адрес. г. Волгодонск, улица  Энтузиастов 9а , Сквер “Дружбы”
Администратор  Светлана +79896260290

## Режим работы

с 10.00-21.30 без перерыва и выходных.

## Бронирование

Для бронирования дня рождения требуется предоплата 1000 рублей (переводом на карту администратора)

# Услуги и локации комплекса

## Игровые комнаты и локации
{'source': 'main_info.md'}
-----
С помощью специальных конфетти. над гостями раскидывается тысячи блестящих лент из фольги. Невесомые серебряные лепестки парят в воздухе под веселую зажигательную музыку.

## Аренда беседки для празднования дня рождения

Отдельная беседка со столом. Можно перекусить и пообщаться компанией. Беседка украшена гирляндой-растяжкой и 10 шариками. На столе - праздничная скатерть. Есть куллер с питьевой водой. 

# Дополнительная информация. Аренда беседки и групповое посещение игровых комнат на время празднования дня рождения

Напитки, торт, еду можно принести с собой.
Нож и хол

In [ ]:
answers = vectordb.max_marginal_relevance_search(question,k=2, fetch_k=5)
printAnswer(answers)

# Общая информация о комплексе

## Контактная информация

Адрес. г. Волгодонск, улица  Энтузиастов 9а , Сквер “Дружбы”
Администратор  Светлана +79896260290

## Режим работы

с 10.00-21.30 без перерыва и выходных.

## Бронирование

Для бронирования дня рождения требуется предоплата 1000 рублей (переводом на карту администратора)

# Услуги и локации комплекса

## Игровые комнаты и локации
{'source': 'main_info.md'}
-----
# Скидки и особые условия

- Многодетные семьи при предъявлении документов получают скидку 50%.
- Особые дети или дети с ограниченными возможностями проходят бесплатно (при предъявлении соответствующих документов).
- Дети из детских домов также проходят бесплатно при предъявлении документов.
{'source': 'main_info.md'}
-----


In [ ]:
!pip install langchainhub

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain import hub

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "твой API-ключ"

retriever = vectordb.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\nФрагмент текста\n\n".join(doc.page_content for doc in docs)

model = ChatOpenAI(
    model="gpt-4o",
    temperature=0.7,
    )

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

rag_chain.invoke("Как с Вами связаться?")

ModuleNotFoundError: No module named 'langchain_openai'

In [ ]:
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

examples = [
    {"input": "2 плюс 2", "output": "равно четыре"},
    {"input": "2 плюс 3", "output": "равно пять"},
    {"input": "Как лает собака", "output": "ГАВ-ГАВ"},
    {"input": "Что говорит хрюша","output": "ХРЮ-ХРЮ",},
]

to_vectorize = [" ".join(example.values()) for example in examples]
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)

In [ ]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

example_selector.select_examples({"input": "2 минус 1"})
example_selector.select_examples({"input": "что говорят рыбы"})

[{'input': 'Что говорит хрюша', 'output': 'ХРЮ-ХРЮ'},
 {'input': 'Как лает собака', 'output': 'ГАВ-ГАВ'}]

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

few_shot_prompt = FewShotChatMessagePromptTemplate(
    input_variables=["input"],
    example_selector=example_selector,
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{input}"), ("ai", "{output}")]
    ),
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

print(few_shot_prompt.invoke(input="2 минус 1"))

messages=[HumanMessage(content='2 плюс 3'), AIMessage(content='равно пять'), HumanMessage(content='2 плюс 2'), AIMessage(content='равно четыре')]


In [ ]:
parser = StrOutputParser()
chain = final_prompt | model | parser

chain.invoke({"input": "2 минус 1"})
chain.invoke({"input": "что говорят куры"})

'Куры говорят "Ко-ко-ко" или "Куд-кудах".'